In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
from keras.datasets import mnist

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
results_path = 'results'

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "bald": bald_info_value
}

In [4]:
class MLP_REG(nn.Module):
    def __init__(self,):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255.
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255.

### Active Learning Procedure

In [6]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [7]:
for exp_iter in range(EXPERIMENT_COUNT):
    np.random.seed(exp_iter)
    initial_idx = np.array([],dtype=int)
    for i in range(10):
        idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
        initial_idx = np.concatenate((initial_idx, idx))
    for func_name, acquisition_func in ACQ_FUNCS.items():  
        X_initial = X_train[initial_idx]
        y_initial = y_train[initial_idx]

        X_pool = np.delete(X_train, initial_idx, axis=0)
        y_pool = np.delete(y_train, initial_idx, axis=0)

        model = MLP_REG().to(DEVICE)

        estimator = NeuralNetClassifier(model,
                                      max_epochs=MAX_EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      lr=LEARNING_RATE,
                                      optimizer=torch.optim.Adam,
                                      criterion=torch.nn.CrossEntropyLoss,
                                      train_split=None,
                                      verbose=0,
                                      device=DEVICE)



        acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                          X_test,
                                                          y_test,
                                                          X_pool,
                                                          y_pool,
                                                          X_initial,
                                                          y_initial,
                                                          estimator,)
        file_name = os.path.join(results_path, func_name + "_exp_" + str(exp_iter) + ".npy")
        np.save(file_name, (acc_arr, dataset_size_arr))

[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32)]
Accuracy after query 1: 0.6009
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32)]
Accuracy after query 2: 0.6242
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=flo

      dtype=float32)]
Accuracy after query 10: 0.7622
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66302913, 0.66163313],
      dtype=float32), arr

      dtype=float32)]
Accuracy after query 15: 0.8126
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66302913, 0.66163313],
      dtype=float32), arr

      dtype=float32)]
Accuracy after query 19: 0.8238
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66302913, 0.66163313],
      dtype=float32), arr

      dtype=float32)]
Accuracy after query 22: 0.8276
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66302913, 0.66163313],
      dtype=float32), arr

...

       1.0277991, 1.0243018, 1.0111275, 0.9911544], dtype=float32)]
Accuracy after query 94: 0.9300
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66

       1.0761313, 1.0616288, 1.048962 , 1.012216 ], dtype=float32)]
Accuracy after query 95: 0.9385
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66

       1.0551299, 1.0241326, 1.0184193, 1.0159937], dtype=float32)]
Accuracy after query 96: 0.9433
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66

       1.0932904, 1.0874377, 1.079909 , 1.0796387], dtype=float32)]
Accuracy after query 97: 0.9376
[array([0.983516  , 0.90871775, 0.9026205 , 0.8922441 , 0.87842035,
       0.8555981 , 0.8551717 , 0.85134536, 0.8509833 , 0.8454261 ],
      dtype=float32), array([0.772651  , 0.73511773, 0.73374546, 0.73126894, 0.7194717 ,
       0.7003572 , 0.69993293, 0.69402915, 0.6875464 , 0.67076445],
      dtype=float32), array([0.7787925 , 0.75205153, 0.7348954 , 0.7010422 , 0.68692625,
       0.67080414, 0.6642042 , 0.6475004 , 0.6458367 , 0.6395575 ],
      dtype=float32), array([0.7310171 , 0.7149032 , 0.7115784 , 0.6881973 , 0.6761898 ,
       0.66774315, 0.6650765 , 0.63165927, 0.63161117, 0.62579   ],
      dtype=float32), array([0.8693786 , 0.76551646, 0.7501515 , 0.7459649 , 0.72247577,
       0.72208625, 0.7215627 , 0.6861701 , 0.6815004 , 0.6599338 ],
      dtype=float32), array([0.7468603 , 0.7316122 , 0.72732276, 0.72343   , 0.7103315 ,
       0.7047857 , 0.7041438 , 0.69350195, 0.66

       1.0836157, 1.0760638, 1.0742028, 1.0659395], dtype=float32)]
Accuracy after query 98: 0.9428
